In [30]:
import torch
import torchmetrics

In [31]:
import pandas as pd
import os
from sampo.scheduler.selection.neural_net import NeuralNetTrainer, NeuralNet, NeuralNetType
from sklearn.preprocessing import StandardScaler

path = os.path.join(os.getcwd(), 'datasets/wg_contractor_dataset_100_objs.csv')
train_dataset = pd.read_csv(path, index_col='index')
for col in train_dataset.columns[:-1]:
    train_dataset[col] = train_dataset[col].apply(lambda x: float(x))

In [32]:
train_dataset['label'] = train_dataset['label'].apply(lambda x: [int(i) for i in x.split()])

In [33]:
train_dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
index,,,,,,,,,,,,,,
0,115.0,0.901729,0.313043,30.930435,7.0,0.008696,0.008696,0.026087,0.330435,0.478261,0.113043,0.026087,0.008696,"[42, 27, 5, 42, 81, 27]"
1,138.0,0.811852,0.355072,29.170290,7.0,0.007246,0.007246,0.021739,0.318841,0.442029,0.173913,0.021739,0.007246,"[47, 31, 5, 47, 92, 31]"
2,122.0,1.042841,0.344262,32.971311,7.0,0.008197,0.008197,0.024590,0.360656,0.418033,0.147541,0.024590,0.008197,"[68, 44, 5, 68, 133, 44]"
3,115.0,0.952107,0.321739,31.969565,7.0,0.008696,0.008696,0.026087,0.339130,0.443478,0.139130,0.026087,0.008696,"[43, 28, 5, 43, 83, 28]"
4,46.0,1.053998,0.586957,33.021739,6.0,0.021739,0.021739,0.043478,0.586957,0.282609,0.021739,0.021739,0.000000,"[40, 26, 5, 40, 78, 26]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,137.0,0.987916,0.321168,32.160584,7.0,0.007299,0.007299,0.021898,0.306569,0.437956,0.189781,0.021898,0.007299,"[54, 35, 5, 54, 105, 35]"
96,214.0,0.808632,0.373832,29.126168,7.0,0.004673,0.004673,0.018692,0.313084,0.481308,0.163551,0.018692,0.004673,"[58, 38, 5, 58, 113, 38]"
97,36.0,1.547846,0.638889,40.277778,6.0,0.027778,0.027778,0.055556,0.638889,0.194444,0.027778,0.027778,0.000000,"[54, 35, 5, 54, 105, 35]"


In [34]:
input_parameters = 13
layer_size = 15
layer_count = 6
out_size = 6
learning_rate = 0.007

model = NeuralNet(input_parameters, layer_size, layer_count, out_size)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scorer = torchmetrics.regression.MeanSquaredError()

trainer = NeuralNetTrainer(model, criterion, optimizer, scorer, 32)

In [35]:
type(train_dataset)

pandas.core.frame.DataFrame

In [15]:
scaler = StandardScaler()
scaler.fit(train_dataset.drop(columns=['label']))
scaled_dataset = scaler.transform(train_dataset.drop(columns=['label']))
scaled_dataset = pd.DataFrame(scaled_dataset, columns=train_dataset.drop(columns=['label']).columns)
train_dataset = pd.concat([scaled_dataset, train_dataset['label']], axis=1)
train_dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
0,-0.112887,-0.490887,-0.788239,-0.442362,0.431484,-0.317028,-0.265376,-0.194649,-0.236057,0.607487,-0.301440,0.587385,0.933373,"[42, 27, 5, 42, 81, 27]"
1,0.360907,-0.848211,-0.446754,-0.887473,0.431484,-0.510602,-0.345482,-0.312215,-0.313941,0.307357,0.536177,-0.416768,0.491294,"[47, 31, 5, 47, 92, 31]"
2,0.031312,0.070136,-0.534586,0.073740,0.431484,-0.383668,-0.292954,-0.235123,-0.033047,0.108582,0.173275,0.241693,0.781182,"[68, 44, 5, 68, 133, 44]"
3,-0.112887,-0.290599,-0.717587,-0.179584,0.431484,-0.317028,-0.265376,-0.194649,-0.177644,0.319362,0.057539,0.587385,0.933373,"[43, 28, 5, 43, 83, 28]"
4,-1.534269,0.114494,1.437305,0.086492,-1.839484,1.425146,0.455577,0.275613,1.487131,-1.013215,-1.557866,-0.416768,-1.719099,"[40, 26, 5, 40, 78, 26]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.340308,-0.148231,-0.722228,-0.131279,0.431484,-0.503538,-0.342559,-0.307924,-0.396373,0.273619,0.754534,-0.380120,0.507429,"[54, 35, 5, 54, 105, 35]"
96,1.926488,-0.861015,-0.294334,-0.898630,0.431484,-0.854334,-0.487726,-0.394621,-0.352610,0.632731,0.393592,-1.120613,-0.293705,"[58, 38, 5, 58, 113, 38]"
97,-1.740266,2.077890,1.859255,1.921419,-1.839484,2.231708,0.789352,0.602185,1.835987,-1.743532,-1.474769,0.977888,-1.719099,"[54, 35, 5, 54, 105, 35]"
98,-1.740266,3.294688,1.633561,3.375494,-1.839484,2.231708,0.789352,0.602185,1.649389,-1.513432,-1.474769,0.977888,-1.719099,"[65, 42, 5, 65, 128, 42]"


In [36]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_dataset.drop(columns=['label']), train_dataset['label'])

In [37]:
from sampo.scheduler.selection.validation import cross_val_score
from sklearn.metrics import accuracy_score

s = cross_val_score(X=x_train,
                    y=y_train,
                    model=trainer,
                    epochs=100,
                    folds=5,
                    shuffle=True,
                    type_task=NeuralNetType.REGRESSION)

Epoch [1/100], Average Loss: 3283.8860
Epoch [2/100], Average Loss: 3272.4633
Epoch [3/100], Average Loss: 3245.6322
Epoch [4/100], Average Loss: 3201.6300
Epoch [5/100], Average Loss: 3110.5658
Epoch [6/100], Average Loss: 2905.2577
Epoch [7/100], Average Loss: 2548.3596
Epoch [8/100], Average Loss: 1968.7076
Epoch [9/100], Average Loss: 1349.8068
Epoch [10/100], Average Loss: 1327.9887
Epoch [11/100], Average Loss: 1044.6762
Epoch [12/100], Average Loss: 683.7042
Epoch [13/100], Average Loss: 578.4877
Epoch [14/100], Average Loss: 487.8139
Epoch [15/100], Average Loss: 327.6452
Epoch [16/100], Average Loss: 278.5255
Epoch [17/100], Average Loss: 303.3887
Epoch [18/100], Average Loss: 227.5749
Epoch [19/100], Average Loss: 208.6012
Epoch [20/100], Average Loss: 229.7920
Epoch [21/100], Average Loss: 206.8055
Epoch [22/100], Average Loss: 180.7632
Epoch [23/100], Average Loss: 199.0041
Epoch [24/100], Average Loss: 171.3696
Epoch [25/100], Average Loss: 144.6870
Epoch [26/100], Average

2023-10-12 12:59:47,755	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [70/100], Average Loss: 86.3832
Epoch [71/100], Average Loss: 84.9662
Epoch [72/100], Average Loss: 85.7414
Epoch [73/100], Average Loss: 85.3628
Epoch [74/100], Average Loss: 85.5347
Epoch [75/100], Average Loss: 85.6914
Epoch [76/100], Average Loss: 85.3526
Epoch [77/100], Average Loss: 85.4710
Epoch [78/100], Average Loss: 84.2442
Epoch [79/100], Average Loss: 84.4181
Epoch [80/100], Average Loss: 87.8093
Epoch [81/100], Average Loss: 85.4733
Epoch [82/100], Average Loss: 84.3529
Epoch [83/100], Average Loss: 92.7867
Epoch [84/100], Average Loss: 85.8525
Epoch [85/100], Average Loss: 84.5843
Epoch [86/100], Average Loss: 87.5026
Epoch [87/100], Average Loss: 84.8398
Epoch [88/100], Average Loss: 83.3778
Epoch [89/100], Average Loss: 84.3621
Epoch [90/100], Average Loss: 86.1120
Epoch [91/100], Average Loss: 86.8025
Epoch [92/100], Average Loss: 90.1473
Epoch [93/100], Average Loss: 86.2607
Epoch [94/100], Average Loss: 86.5683
Epoch [95/100], Average Loss: 87.4755
Epoch [96/10

2023-10-12 12:59:48,163	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [74/100], Average Loss: 77.3472
Epoch [75/100], Average Loss: 74.8206
Epoch [76/100], Average Loss: 78.9359
Epoch [77/100], Average Loss: 75.7585
Epoch [78/100], Average Loss: 76.3189
Epoch [79/100], Average Loss: 74.6604
Epoch [80/100], Average Loss: 77.6995
Epoch [81/100], Average Loss: 83.3102
Epoch [82/100], Average Loss: 79.0176
Epoch [83/100], Average Loss: 75.7706
Epoch [84/100], Average Loss: 82.0960
Epoch [85/100], Average Loss: 73.7336
Epoch [86/100], Average Loss: 81.9187
Epoch [87/100], Average Loss: 75.7803
Epoch [88/100], Average Loss: 78.5866
Epoch [89/100], Average Loss: 77.6544
Epoch [90/100], Average Loss: 75.3214
Epoch [91/100], Average Loss: 90.6884
Epoch [92/100], Average Loss: 82.6620
Epoch [93/100], Average Loss: 85.3454
Epoch [94/100], Average Loss: 82.2631
Epoch [95/100], Average Loss: 72.6090
Epoch [96/100], Average Loss: 84.5086
Epoch [97/100], Average Loss: 71.9454
Epoch [98/100], Average Loss: 83.8803
Epoch [99/100], Average Loss: 75.0441
Epoch [100/1

...

Epoch [73/100], Average Loss: 84.6776
Epoch [74/100], Average Loss: 77.8772
Epoch [75/100], Average Loss: 74.4968
Epoch [76/100], Average Loss: 76.6297
Epoch [77/100], Average Loss: 60.3831
Epoch [78/100], Average Loss: 73.2085
Epoch [79/100], Average Loss: 64.2968
Epoch [80/100], Average Loss: 64.2377
Epoch [81/100], Average Loss: 69.4933
Epoch [82/100], Average Loss: 59.8596
Epoch [83/100], Average Loss: 66.6536
Epoch [84/100], Average Loss: 65.0913
Epoch [85/100], Average Loss: 60.0239
Epoch [86/100], Average Loss: 71.7974
Epoch [87/100], Average Loss: 78.6015
Epoch [88/100], Average Loss: 61.3856
Epoch [89/100], Average Loss: 71.3982
Epoch [90/100], Average Loss: 57.5129
Epoch [91/100], Average Loss: 62.8964
Epoch [92/100], Average Loss: 57.0701
Epoch [93/100], Average Loss: 57.9866
Epoch [94/100], Average Loss: 56.4372
Epoch [95/100], Average Loss: 63.8218
Epoch [96/100], Average Loss: 57.5541
Epoch [97/100], Average Loss: 58.4155
Epoch [98/100], Average Loss: 58.0937
Epoch [99/10

2023-10-12 12:59:48,988	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [79/100], Average Loss: 52.7594
Epoch [80/100], Average Loss: 55.8416
Epoch [81/100], Average Loss: 45.7440
Epoch [82/100], Average Loss: 53.4095
Epoch [83/100], Average Loss: 50.3921
Epoch [84/100], Average Loss: 51.8713
Epoch [85/100], Average Loss: 56.2956
Epoch [86/100], Average Loss: 60.7181
Epoch [87/100], Average Loss: 48.3878
Epoch [88/100], Average Loss: 58.6028
Epoch [89/100], Average Loss: 50.9777
Epoch [90/100], Average Loss: 48.2478
Epoch [91/100], Average Loss: 48.5466
Epoch [92/100], Average Loss: 49.3376
Epoch [93/100], Average Loss: 54.6049
Epoch [94/100], Average Loss: 45.1911
Epoch [95/100], Average Loss: 50.3543
Epoch [96/100], Average Loss: 49.6365
Epoch [97/100], Average Loss: 51.7078
Epoch [98/100], Average Loss: 63.7571
Epoch [99/100], Average Loss: 72.0100
Epoch [100/100], Average Loss: 61.1766
Epoch [1/100], Average Loss: 53.9199
Epoch [2/100], Average Loss: 60.6693
Epoch [3/100], Average Loss: 62.4540
Epoch [4/100], Average Loss: 67.7065
Epoch [5/100], 

2023-10-12 12:59:49,431	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [100/100], Average Loss: 51.8744


In [38]:
index = [-10000] * input_parameters
for fold in list(model.parameters())[0]:
    for i in range(len(fold)):
        index[i] = max(index[i], fold[i])
for i in range(len(index)):
    print(i, index[i])

0 tensor(0.2189, grad_fn=<SelectBackward0>)
1 tensor(1.6328, grad_fn=<SelectBackward0>)
2 tensor(2.8139, grad_fn=<SelectBackward0>)
3 tensor(0.4015, grad_fn=<SelectBackward0>)
4 tensor(0.8049, grad_fn=<SelectBackward0>)
5 tensor(3.7189, grad_fn=<SelectBackward0>)
6 tensor(1.6510, grad_fn=<SelectBackward0>)
7 tensor(1.9515, grad_fn=<SelectBackward0>)
8 tensor(2.3776, grad_fn=<SelectBackward0>)
9 tensor(0.3425, grad_fn=<SelectBackward0>)
10 tensor(0.9870, grad_fn=<SelectBackward0>)
11 tensor(1.2404, grad_fn=<SelectBackward0>)
12 tensor(1.6748, grad_fn=<SelectBackward0>)


In [39]:
print(s)

(239.33650716145834, 7.479265848795573, <sampo.scheduler.selection.neural_net.NeuralNetTrainer object at 0x000001B47F467CA0>)


In [40]:
test_dataset = x_test

In [41]:
predicted = trainer.predict([torch.Tensor(v) for v in test_dataset.iloc[:, :].values])
array = []
label_test = y_test.to_numpy()
for i in range(len(predicted)):
    array.append(sum((predicted[i] - label_test[i]) ** 2))
(sum(array) / len(array)) ** (1 / 2)

15.86457572932663

In [45]:
predicted

array([[ 57.3298   ,  37.07157  ,   5.1664343,  57.324955 , 111.55159  ,
         37.04023  ],
       [ 61.66734  ,  39.87013  ,   5.520999 ,  61.64526  , 119.96209  ,
         39.843422 ],
       [ 56.182514 ,  36.330105 ,   5.0719843,  56.181637 , 109.32636  ,
         36.2983   ],
       [ 60.230453 ,  38.932423 ,   5.397867 ,  60.209064 , 117.17043  ,
         38.910656 ],
       [ 54.911392 ,  35.518536 ,   4.9726486,  54.91959  , 106.866066 ,
         35.480156 ],
       [ 66.93181  ,  43.22555  ,   5.929315 ,  66.86941  , 130.14854  ,
         43.229595 ],
       [ 68.20225  ,  44.065914 ,   6.0442486,  68.14454  , 132.62263  ,
         44.058666 ],
       [ 51.52979  ,  33.341614 ,   4.698829 ,  51.553715 , 100.311646 ,
         33.29665  ],
       [ 65.51152  ,  42.340874 ,   5.8301754,  65.46968  , 127.411    ,
         42.32404  ],
       [ 48.04828  ,  31.110708 ,   4.4224205,  48.093285 ,  93.56901  ,
         31.052694 ],
       [ 59.28066  ,  38.324837 ,   5.3230095,  59

In [43]:
label_test


array([list([63, 41, 5, 63, 124, 41]), list([59, 38, 5, 59, 116, 38]),
       list([57, 37, 5, 57, 112, 37]), list([61, 40, 5, 61, 120, 40]),
       list([56, 36, 5, 56, 110, 36]), list([56, 36, 5, 56, 109, 36]),
       list([66, 43, 5, 66, 129, 43]), list([58, 38, 5, 58, 113, 38]),
       list([53, 34, 5, 53, 104, 34]), list([40, 26, 5, 40, 77, 26]),
       list([57, 37, 5, 57, 111, 37]), list([58, 37, 5, 58, 113, 37]),
       list([50, 32, 5, 50, 97, 32]), list([31, 20, 5, 31, 60, 20]),
       list([53, 34, 5, 53, 102, 34]), list([65, 42, 5, 65, 126, 42]),
       list([65, 42, 5, 65, 127, 42]), list([48, 31, 5, 48, 92, 31]),
       list([65, 42, 5, 65, 128, 42]), list([49, 32, 5, 49, 96, 32]),
       list([44, 28, 5, 44, 85, 28]), list([47, 31, 5, 47, 92, 31]),
       list([41, 26, 5, 41, 80, 26]), list([55, 36, 5, 55, 108, 36]),
       list([49, 32, 5, 49, 95, 32])], dtype=object)